In [1]:
#General libraries
import numpy as np
import pandas as pd
import sys
import os 
import re
from dateutil.parser import parse
import datetime 
import json

#Libraries for parsing and getting text from websites
from codecs import xmlcharrefreplace_errors
import feedparser
import urllib.parse
import requests
from bs4 import BeautifulSoup
import ssl

#Libraries for coref
import spacy
import crosslingual_coreference
from crosslingual_coreference import Predictor
import en_core_web_sm

#Loading extras for parsing
ssl._create_default_https_context = ssl._create_unverified_context #avoiding SSL errors
headers =  {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0"} #avoiding some bot-shields
predictor = Predictor(language="en_core_web_sm", device=-1, model_name="spanbert")

/home/werner/thesis_valentin/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package omw-1.4 to /home/werner/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
df = pd.read_csv("articles_url.csv")

In [3]:
res = []
for row in df[(df.paper != "nyt") & (df.paper != "jt")].iterrows():
    #skip nyt and jt because not scrapable without java
    response = requests.get(row[1]["url"], headers = headers) 
    soup = BeautifulSoup(response.content, "html.parser")

    if row[1]["paper"] == "f24":
        try:
            res.append([row[1]["paper"],row[1]["url"],soup.find("div", "t-content__body u-clearfix").text.replace("\n","")])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")
    elif row[1]["paper"] == "cbc":
        try:
            res.append([row[1]["paper"],row[1]["url"],soup.find("div", "story").text])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")
    elif row[1]["paper"] == "it":
        try:
            res.append([row[1]["paper"],row[1]["url"],soup.find("div", "_3YYSt clearfix").text.replace("\n","")])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")
    elif row[1]["paper"] == "smh":
        try:
            res.append([row[1]["paper"],row[1]["url"],soup.find("div", "_1665V _2q-Vk").text.replace("\n","")])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")
    elif row[1]["paper"] == "bbc":
        try:
            content = soup.find_all("div", "ssrcss-11r1m41-RichTextComponentWrapper ep2nwvo0")
            text = ""
            for para in content:
                text = text + " " + para.text
            res.append([row[1]["paper"],row[1]["url"],text.replace("\n","")])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")
    elif row[1]["paper"] == "spiegel":
        try:
            content = soup.find_all("div", "RichText lg:w-8/12 md:w-10/12 lg:mx-auto md:mx-auto lg:px-24 md:px-24 sm:px-16 break-words word-wrap")
            text = ""
            for para in content:
                text = text + " " + para.text
            res.append([row[1]["paper"],row[1]["url"],text.replace("\n", " ")])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")
    elif row[1]["paper"] == "tass":
        try:
            res.append([row[1]["paper"],row[1]["url"],soup.find("div", "text-block").text.replace("\n","")])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")
    elif row[1]["paper"] == "folha":
        try:
            res.append([row[1]["paper"],row[1]["url"],soup.find("div", "c-news__content").text.replace("\n","")])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")
    elif row[1]["paper"] == "bat":
        try:
            res.append([row[1]["paper"],row[1]["url"],soup.find("div", "col-12 px-4").text.replace("\n","")])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")
    elif row[1]["paper"] == "independent":
        try:
            content = soup.find_all("p", "paragraph inline-placeholder")
            text = ""
            for para in content:
                text = text + " " + para.text
            res.append([row[1]["paper"],row[1]["url"],text.replace("\n", " ")])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")
    elif row[1]["paper"] == "ewn":
        try:
            res.append([row[1]["paper"],row[1]["url"],". ".join(soup.find("div", "medium-12 columns").text.replace("\n","").split("."))])
        except: 
            print(f"couldn't scrape {row[1]['title']} from {row[1]['paper']}")

couldn't scrape Police chief calls response to South Korea crowd crush "inadequate" from smh
couldn't scrape ‘Starlink is the difference’: Internet connection gives Ukraine edge in drone war from f24
couldn't scrape Russian president accused of "death by freezing" from smh
couldn't scrape Police chief calls response to South Korea crowd crush "inadequate" from smh
couldn't scrape Man charged with assault and attempted kidnapping after Pelosi attack from smh
couldn't scrape Police apologise for handling of South Korea Halloween crowd crush from smh
couldn't scrape Wave of Russian missiles target key infrastructure sites in Kyiv from smh
couldn't scrape Sydney woman killed in Seoul crowd crush from smh
couldn't scrape Lula da Silva victorious in Brazilian election from smh
couldn't scrape Gallery: The best photos from around the world from smh
couldn't scrape Russian president accused of "death by freezing" from smh
couldn't scrape 'Shark Tank India' locks the sharks for its second seaso

In [5]:
def run_coref(new_df):
    """applies coreference resolution to the new scraped articles"""
    #load spanbert model, as it is currently one of the state of the art models and achieved best performance on the data
    predictor = Predictor(language="en_core_web_sm", device=-1, model_name="spanbert")
    #apply coreference resolution
    new_df["coref_text"] = new_df.full_text.apply(lambda x: predictor.predict(str(x))["resolved_text"])
    
    return new_df

In [4]:
new_df = pd.DataFrame(res, columns = ["paper", "link", "full_text"])

In [4]:
import torch
predictor = Predictor(language="en_core_web_sm", device=-1, model_name="spanbert")

In [2]:
df = pd.read_csv("intermediate_full_articles-pre26-11.csv")

In [3]:
coref = []
for row in df.iterrows():
    try: coref.append(predictor.predict(row[1]["full_text"])["resolved_text"])
    except: coref.append(row[1]["full_text"])

In [12]:
new_df["text"] = coref

In [14]:
new_df[["paper","link","text"]].to_csv("articles_url_coref.csv")